In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [18]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [19]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [20]:
not_ran = [208, 343, 273, 372, 346, 380, 243, 45, 260, 75]

In [24]:
d3 = [273, 372, 221]

In [25]:
pbar = tqdm()

0it [00:00, ?it/s]

In [26]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in d3:
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
        rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_02_08_pm_peak.html')
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        print(e)

calculating for agency: 273...
found parquet
found parquet
found parquet
found parquet
found_parquet
could not generate delays for trip 992065
Columns must be same length as key
could not generate delays for trip 991528
Columns must be same length as key
could not generate delays for trip 991792
zero-size array to reduction operation maximum which has no identity
could not generate delays for trip 992869
zero-size array to reduction operation maximum which has no identity
could not generate delays for trip 993984
Columns must be same length as key
could not generate delays for trip 989947
Columns must be same length as key
could not generate delays for trip 993942
Columns must be same length as key
could not generate delays for trip 991260
zero-size array to reduction operation maximum which has no identity
could not generate delays for trip 991270
Columns must be same length as key
could not generate delays for trip 991267
Columns must be same length as key
could not generate delays f

TopologyException: assigned depths do not match at -121987.84282226175 62752.234451979428
/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:503: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



complete for agency: 273
calculating for agency: 372...
found parquet
found parquet
found parquet
found parquet
found_parquet
could not generate delays for trip 4309260b-9393-4256-9f0a-faac39945792
zero-size array to reduction operation maximum which has no identity
could not generate delays for trip 2385d28f-6ff4-4b37-88be-c3ea9dee8aa5
Columns must be same length as key
could not generate delays for trip 512c199f-e125-400e-a175-057e44eabdf8
Columns must be same length as key
could not generate delays for trip 5be5b137-f163-4efc-ab68-7bcf8411eb80
Columns must be same length as key
could not generate delays for trip a0422399-92a4-45f5-9b5e-83e73f83af16
time data '24:00:04' does not match format '%H:%M:%S'
could not generate delays for trip a4ae9c70-4dfa-4bbe-8a03-932609140ee7
time data '24:48:31' does not match format '%H:%M:%S'
could not generate delays for trip 6e0eca52-bc50-4417-aaf0-4c2cfad434f8
Columns must be same length as key
could not generate delays for trip 35fdc864-ebbb-4a4c

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:503: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



complete for agency: 372
calculating for agency: 221...
gs://calitp-analytics-data/data-analyses/rt_delay/cached_views/
stops_221_2022-02-08


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


found_parquet
vehicle positions gdf must not be empty
could not generate delays for trip t_1523700_b_30719_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523715_b_30719_tn_2
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523698_b_30719_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523715_b_30719_tn_1
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523713_b_30719_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523711_b_30719_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523715_b_30719_tn_3
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523697_b_30719_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1523712_b_30719_tn_0
strptime() argument 1 must be str, not None
could not generate delays fo

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:503: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [27]:
air_joined['url'] = air_joined.apply(lambda x:
                    f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
                                    axis = 1)

# air_joined.to_csv('linked.csv')

In [28]:
air_joined

,Name,calitp_itp_id,Reporting Category,Caltrans District,GTFS Static Status,rt_status,Missing Static,Services Needing Alerts,Services Needing TripUpdates or VehiclePositions,calitp_url_number,min_timestamp,max_timestamp,url
0,Long Beach Transit,170,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,AquaLink,"Galaxy Express, AquaLink, UCLA/Westwood Commut...","Galaxy Express, AquaLink, UCLA/Westwood Commut...",0,2021-07-08 20:58:59,2022-01-14 12:30:50,http://docs.calitp.org/data-analyses/rt_delay/...
1,Peninsula Corridor Joint Powers Board,246,Core,04 - Oakland,Static Incomplete,RT Incomplete,"Caltrain Shuttles, Caltrain Millbrae Express","Caltrain Shuttles, Caltrain Millbrae Express","Caltrain Shuttles, Caltrain Millbrae Express",1,2021-12-29 14:34:19,2022-01-14 12:31:00,http://docs.calitp.org/data-analyses/rt_delay/...
2,City of Visalia,361,Core,06 - Fresno,Static Incomplete,RT Incomplete,V-LINE,"V-LINE, Sequoia Shuttle, Visalia Dial-A-Ride","V-LINE, Sequoia Shuttle, Visalia Dial-A-Ride",0,2015-01-03 02:55:42,2022-01-14 12:31:07,http://docs.calitp.org/data-analyses/rt_delay/...
3,San Mateo County Transit District,290,Core,04 - Oakland,Static Incomplete,RT OK,"Bayshore Brisbane Senior Shuttle, SamTrans Shu...","Redi-Wheels, RediCoast, Bayshore Brisbane Seni...","Redi-Wheels, RediCoast, Bayshore Brisbane Seni...",1,2021-12-29 14:34:05,2022-01-14 12:30:58,http://docs.calitp.org/data-analyses/rt_delay/...
4,Marin County Transit District,194,Core,04 - Oakland,Static Incomplete,RT OK,Muir Woods Shuttle,"Marin Access Paratransit, Muir Woods Shuttle, ...","Marin Access Paratransit, Muir Woods Shuttle, ...",0,2015-01-01 16:55:46,2022-01-14 12:31:08,http://docs.calitp.org/data-analyses/rt_delay/...
5,Monterey-Salinas Transit,208,Core,05 - San Luis Obispo,Static Incomplete,RT OK,MST Senior Shuttles,"MST Monterey Trolley, MST RIDES, MST Taxi Vouc...","MST Monterey Trolley, MST RIDES, MST Taxi Vouc...",0,2021-07-08 20:57:45,2022-01-14 12:31:02,http://docs.calitp.org/data-analyses/rt_delay/...
6,Napa Valley Transportation Authority,218,Core,04 - Oakland,Static OK,RT Incomplete,None,"Calistoga Shuttle, Taxi Scrip, Vine Go, Americ...","Calistoga Shuttle, Taxi Scrip, Vine Go, Americ...",1,2021-12-29 14:34:17,2022-01-14 12:31:07,http://docs.calitp.org/data-analyses/rt_delay/...
7,"Golden Gate Bridge, Highway and Transportation...",127,Core,04 - Oakland,Static OK,RT Incomplete,None,"Golden Gate Ferry, Golden Gate Transit Paratra...","Golden Gate Ferry, Golden Gate Transit Paratra...",1,2021-12-29 14:34:12,2022-01-14 12:30:59,http://docs.calitp.org/data-analyses/rt_delay/...
8,Transit Joint Powers Authority for Merced County,343,Core,10 - Stockton,Static OK,RT Incomplete,None,"Merced The Micro Bus, Merced The Bus Paratransit","Merced The Bus, Merced The Micro Bus, Merced T...",0,2021-12-21 13:50:26,2022-01-14 12:31:08,http://docs.calitp.org/data-analyses/rt_delay/...
9,Sacramento Regional Transit District,273,Core,03 - Marysville,Static OK,RT Incomplete,None,"SacRT Light Rail, e-van, SacRT GO, Elk Grove T...","e-van, SacRT GO, SacRT Neighborhood Ride",0,2021-12-21 14:04:00,2022-01-14 12:31:00,http://docs.calitp.org/data-analyses/rt_delay/...
